<font style='font-size:1.5em'>**🧑‍🏫 Week 07 Lecture</font><br>
<font style='font-size:1.3em;color:#888888'>Normalising JSON + the Groupby -> Apply -> Combine Strategy** </font>

<font style='font-size:1.2em'>LSE [DS105A](https://lse-dsi.github.io/DS105/autumn-term/index.html){style="color:#e26a4f;font-weight:bold"} – Data for Data Science (2024/25) </font>



<div style="color: #333333; background-color:rgba(226, 106, 79, 0.075); border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); padding: 20px 0 20px 10px; margin: 10px 0 10px 0; flex: 1 1 calc(45% - 20px);min-width: 250px;max-width: 350px;align-items:top;min-height: calc(45% - 20px); box-sizing: border-box;font-size:0.9em;">

🗓️ **DATE:** 14 November 2024 

⌚ **TIME:** 16.00-18.00

📍 **LOCATION:** CLM.5.02
</div>


**AUTHORS:**  Dr. [Jon Cardoso-Silva](https://jonjoncardoso.github.io){style="color:#e26a4f;font-weight:bold"}

**DEPARTMENT:** [LSE Data Science Institute](https://lse.ac.uk/dsi){style="color:#e26a4f;font-weight:bold"}

**OBJECTIVE**: Demonstrate how to 'disentangle' complex JSON data structures using the `json_normalize` function from the `pandas` library and introduce the `groupby -> apply -> combine` strategy to process data in a more efficient way than using loops. We will also discuss the `explode` function to handle cases when we find ourselves with columns made out of lists.

**REFERENCES:**

- The [`pd.json_normalize()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) to convert JSON data more easily into tabular format
- The [DataFrame.explode()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html) function to handle cases when columns are made out of lists
- The [DataFrame.groupby()](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) function, combined with apply() and agg() to aggregate data

---

In [1]:
import json

import pandas as pd

# 1. The problem of nested JSON data

You spent the past few weeks playing with data collected from OpenMeteo, a (mostly) free API that provides weather data for any location in the world in the format of JSON. JSON is indeed the preferred format for APIs, as it is easy to read and write by both humans and machines and easy to be parsed by any programming language.

👉 However, most data analysis libraries, such as `pandas` in Python or `dplyr` in R or SQL, as well as most visualisation libraries, are designed to work with tabular data. This means that we need to convert JSON data into a tabular format to be able to analyse it.

[🤔 **Think about it:** What is the easiest way to convert JSON data into a tabular format using Python?]{style="display:block;background-color:rgba(93, 158, 188, 0.1);padding:0.5em;font-size:1.05em;margin-left:0em;margin-bottom:1em;border-radius:0.5em;"}

To answer that question, let's first understand the structure of the JSON data we are working with. I will use OpenMeteo first as an example. 

## 1.1 Let's start with a familiar example

OpenMeteo already has a fairly straightforward JSON structure. Sure, there were a few nested dictionaries and lists, but nothing you couldn't handle with a few loops and list comprehensions. 

<!-- <details style="border: 1px solid #aaa;border-radius: 4px;padding: .5em .5em 0;"><summary style="    font-weight: bold;margin: -.5em -.5em 0;padding: .5em;">Click here to see the full JSON output from OpenMeteo</summary> -->

What follows below is the response I get after requesting historical weather data using the following parameters:

- London's latitude and longitude
- Period: from 26/Oct until 9/Nov
- Frequency: daily (only)
- Weather variables: weather code, daylight duration, sunshine duration

I copied the output and stored it as a dictionary to save us the hassle of sending a request:

In [2]:
json_output = {
  "latitude": 51.493847,
  "longitude": -0.1630249,
  "generationtime_ms": 0.154972076416016,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 23,
  "daily_units": {
    "time": "iso8601",
    "weather_code": "wmo code",
    "daylight_duration": "s",
    "sunshine_duration": "s"
  },
  "daily": {
    "time": [
      "2024-10-26",
      "2024-10-27",
      "2024-10-28",
      "2024-10-29",
      "2024-10-30",
      "2024-10-31",
      "2024-11-01",
      "2024-11-02",
      "2024-11-03",
      "2024-11-04",
      "2024-11-05",
      "2024-11-06",
      "2024-11-07",
      "2024-11-08",
      "2024-11-09"
    ],
    "weather_code": [3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51, 3],
    "daylight_duration": [35989.18, 35766.05, 35543.86, 35322.81, 35103.12, 34885.01, 34668.7, 34454.44, 34242.45, 34032.99, 33826.3, 33622.64, 33422.29, 33225.15, 33029.27],
    "sunshine_duration": [14002.7, 31127.43, 5811.8, 10787.3, 19920.14, 14421.7, 14437.86, 0, 1098.29, 8162.58, 17941.69, 536.17, 4472.84, 0, 7173.36]
  }
}

If I have a Python dictionary where there aren't any nested dictionaries or lists, one where each key corresponds to a column and the values are lists of the same length, I can build a DataFrame easily using the `pd.DataFrame()` function. 

In [7]:
# To prove that, let's pass just the 'daily' key to the DataFrame constructor:
pd.DataFrame(json_output['daily'])

,time,weather_code,daylight_duration,sunshine_duration
0,2024-10-26,3,35989.18,14002.70
1,2024-10-27,51,35766.05,31127.43
2,2024-10-28,51,35543.86,5811.80
3,2024-10-29,51,35322.81,10787.30
4,2024-10-30,3,35103.12,19920.14
5,2024-10-31,3,34885.01,14421.70
6,2024-11-01,3,34668.70,14437.86
7,2024-11-02,51,34454.44,0.00
8,2024-11-03,3,34242.45,1098.29
9,2024-11-04,3,34032.99,8162.58


☝️ **This works super well when the data is 'flat' and doesn't have nested structures.**

But I lose the other information that is outside the `daily` key such as the `latitude`, `longitude`, `timezone`, etc.

## 1.2 What if I use the DataFrame at the top level?

In other words, what if instead of creating a DataFrame just for the `daily` key, I create a DataFrame for the entire JSON response?

In [12]:
pd.DataFrame(json_output)

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,daily_units,daily
time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,"[2024-10-26, 2024-10-27, 2024-10-28, 2024-10-2..."
weather_code,51.493847,-0.163025,0.154972,0,GMT,GMT,23,wmo code,"[3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51..."
daylight_duration,51.493847,-0.163025,0.154972,0,GMT,GMT,23,s,"[35989.18, 35766.05, 35543.86, 35322.81, 35103..."
sunshine_duration,51.493847,-0.163025,0.154972,0,GMT,GMT,23,s,"[14002.7, 31127.43, 5811.8, 10787.3, 19920.14,..."


📢 **IMPORTANT!** Notice what happens:

- The DataFrame has for rows, one per weather variable (+ time)
- The weather variables (+ time) are stored in the index, not as columns
- The columns are the keys of the **top-level** dictionary

Is this suitable? It depends on what you want to do with the data.

Because we just have a single location (London's latitude and longitude), this repetition of the location information is redundant, wasteful. 


**We do not want wasteful dataframes in this course! We want to live in a world where data is clean, tidy, economical, and efficient!** 

In that case, we can conclude that we just care about the `daily` key and we can discard the rest of the information. Here's how you would do that after creating the DataFrame above:

In [42]:
selected_columns = ['daily']

pd.DataFrame(json_output)[selected_columns]

,daily
time,"[2024-10-26, 2024-10-27, 2024-10-28, 2024-10-2..."
weather_code,"[3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51..."
daylight_duration,"[35989.18, 35766.05, 35543.86, 35322.81, 35103..."
sunshine_duration,"[14002.7, 31127.43, 5811.8, 10787.3, 19920.14,..."


## 1.3 Rows become columns and columns become rows

Notice that the index of each row would make really nice column names? Transpose the DataFrame to achieve that:

In [56]:
# You can use the transpose method to swap the rows and columns:
pd.DataFrame(json_output)[selected_columns].transpose()

,time,weather_code,daylight_duration,sunshine_duration
daily,"[2024-10-26, 2024-10-27, 2024-10-28, 2024-10-2...","[3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51...","[35989.18, 35766.05, 35543.86, 35322.81, 35103...","[14002.7, 31127.43, 5811.8, 10787.3, 19920.14,..."


In [57]:
# Or just use T if you prefer:
pd.DataFrame(json_output)[selected_columns].T

,time,weather_code,daylight_duration,sunshine_duration
daily,"[2024-10-26, 2024-10-27, 2024-10-28, 2024-10-2...","[3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51...","[35989.18, 35766.05, 35543.86, 35322.81, 35103...","[14002.7, 31127.43, 5811.8, 10787.3, 19920.14,..."


## 1.4 The `explode()` function will be your friend

Now, notice that we ended up with a DataFrame where the columns are made out of lists. You will inevitably find yourself in this situation when working with JSON data every now and then.

The `DataFrame.explode()` method is a great way to handle this situation. It expects to receive the column name (as a string) or names (as a list of strings) that you want to explode out into separate rows.

In [76]:
selected_columns = ['daily']
columns_to_explode = ['time', 'weather_code', 'daylight_duration', 'sunshine_duration']

# Explode the columns that contain lists:
pd.DataFrame(json_output)[selected_columns].T.explode(columns_to_explode)

,time,weather_code,daylight_duration,sunshine_duration
daily,2024-10-26,3,35989.18,14002.7
daily,2024-10-27,51,35766.05,31127.43
daily,2024-10-28,51,35543.86,5811.8
daily,2024-10-29,51,35322.81,10787.3
daily,2024-10-30,3,35103.12,19920.14
daily,2024-10-31,3,34885.01,14421.7
daily,2024-11-01,3,34668.7,14437.86
daily,2024-11-02,51,34454.44,0
daily,2024-11-03,3,34242.45,1098.29
daily,2024-11-04,3,34032.99,8162.58


## 1.5 Let me complicate things a bit

What if the JSON output was made of London **and Paris** data? Same variables, same period, same frequency, just two locations.

In [60]:
json_output_two_cities = [
  {
    "latitude": 51.493847,
    "longitude": -0.1630249,
    "generationtime_ms": 0.200033187866211,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 23,
    "daily_units": {
      "time": "iso8601",
      "weather_code": "wmo code",
      "daylight_duration": "s",
      "sunshine_duration": "s"
    },
    "daily": {
      "time": [
        "2024-10-26",
        "2024-10-27",
        "2024-10-28",
        "2024-10-29",
        "2024-10-30",
        "2024-10-31",
        "2024-11-01",
        "2024-11-02",
        "2024-11-03",
        "2024-11-04",
        "2024-11-05",
        "2024-11-06",
        "2024-11-07",
        "2024-11-08",
        "2024-11-09"
      ],
      "weather_code": [3, 51, 51, 51, 3, 3, 3, 51, 3, 3, 3, 3, 3, 51, 3],
      "daylight_duration": [35989.18, 35766.05, 35543.86, 35322.81, 35103.12, 34885.01, 34668.7, 34454.44, 34242.45, 34032.99, 33826.3, 33622.64, 33422.29, 33225.15, 33029.27],
      "sunshine_duration": [14002.7, 31127.43, 5811.8, 10787.3, 19920.14, 14421.7, 14437.86, 0, 1098.29, 8162.58, 17941.69, 536.17, 4472.84, 0, 7173.36]
    }
  },
  {
    "latitude": 48.822495,
    "longitude": 2.2881355,
    "generationtime_ms": 0.160098075866699,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 43,
    "location_id": 1,
    "daily_units": {
      "time": "iso8601",
      "weather_code": "wmo code",
      "daylight_duration": "s",
      "sunshine_duration": "s"
    },
    "daily": {
      "time": [
        "2024-10-26",
        "2024-10-27",
        "2024-10-28",
        "2024-10-29",
        "2024-10-30",
        "2024-10-31",
        "2024-11-01",
        "2024-11-02",
        "2024-11-03",
        "2024-11-04",
        "2024-11-05",
        "2024-11-06",
        "2024-11-07",
        "2024-11-08",
        "2024-11-09"
      ],
      "weather_code": [55, 53, 3, 3, 3, 3, 3, 51, 3, 3, 3, 3, 3, 3, 3],
      "daylight_duration": [36679.21, 36477.63, 36276.98, 36077.46, 35879.25, 35682.56, 35487.58, 35294.54, 35103.64, 34915.11, 34729.17, 34546.06, 34366.02, 34188.94, 34013.09],
      "sunshine_duration": [17721.39, 1329.73, 14961.37, 3869.25, 13851.97, 17390.9, 130.73, 0, 17079.12, 17671.44, 0, 9888.03, 0, 0, 10666.35]
    }
  }
]

In [61]:
# First of all, this isn't even a dictionary but a list
type(json_output_two_cities)

list

What happens when I try to create a DataFrame for the entire JSON response?

In [62]:
pd.DataFrame(json_output_two_cities)

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,daily_units,daily,location_id
0,51.493847,-0.163025,0.200033,0,GMT,GMT,23,"{'time': 'iso8601', 'weather_code': 'wmo code'...","{'time': ['2024-10-26', '2024-10-27', '2024-10...",NaN
1,48.822495,2.288136,0.160098,0,GMT,GMT,43,"{'time': 'iso8601', 'weather_code': 'wmo code'...","{'time': ['2024-10-26', '2024-10-27', '2024-10...",1.0


⚡ **OH NO!** This time around the code we built above will not work. When I select just the `daily` column, I no longer have those neat index names that I can use as column names:

In [88]:
# If you try to transpose after this line, things will get messy
pd.DataFrame(json_output_two_cities)[selected_columns]

,daily
0,"{'time': ['2024-10-26', '2024-10-27', '2024-10..."
1,"{'time': ['2024-10-26', '2024-10-27', '2024-10..."


If your Internet is down and you can't look up any documentation, Google or ChatGPT, then you can always take the long way around and use `for` loops to rebuild the DataFrame.

After all, in case of panic, **DataFrames can always be converted to a mix of list and dictionaries**:

In [118]:
# In case of panic, use `to_dict()` to convert any DataFrame to a dictionary
# Use orient='records' to get a list of dictionaries (easier to read)
list_of_dicts = pd.DataFrame(json_output_two_cities)[selected_columns].to_dict(orient='records')

# Print it out to see the structure

If you remember the concept of list comprehensions (a `for` loop in a single line), you can use them to build a list of DataFrames, one for each location:

In [136]:
# For each list, build a distinct DataFrame
list_of_dfs = [pd.DataFrame(d['daily']) for d in list_of_dicts]

# Use position 0 for London and 1 for Paris
list_of_dfs[0]

,time,weather_code,daylight_duration,sunshine_duration
0,2024-10-26,3,35989.18,14002.70
1,2024-10-27,51,35766.05,31127.43
2,2024-10-28,51,35543.86,5811.80
3,2024-10-29,51,35322.81,10787.30
4,2024-10-30,3,35103.12,19920.14
5,2024-10-31,3,34885.01,14421.70
6,2024-11-01,3,34668.70,14437.86
7,2024-11-02,51,34454.44,0.00
8,2024-11-03,3,34242.45,1098.29
9,2024-11-04,3,34032.99,8162.58


# 2. Meet the `json_normalize()` function

In [160]:
(
    pd.json_normalize(json_output_two_cities)
        .explode(['daily.time', 'daily.weather_code', 'daily.daylight_duration', 'daily.sunshine_duration'])
        .sort_values(['daily.time', 'latitude', 'longitude'])
        .set_index(['daily.time', 'latitude', 'longitude'])
)

generationtime_ms  utc_offset_seconds  \
daily.time latitude  longitude                                          
2024-10-26 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-10-27 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-10-28 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-10-29 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-10-30 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-10-31 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-01 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-02 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-03 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-04 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-05 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-06 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-07 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-08 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   
2024-11-09 48.822495  2.288136           0.160098                   0   
           51.493847 -0.163025           0.200033                   0   

                               timezone timezone_abbreviation  elevation  \
daily.time latitude  longitude                                             
2024-10-26 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-10-27 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-10-28 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-10-29 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-10-30 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-10-31 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-11-01 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-11-02 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-11-03 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-11-04 48.822495  2.288136      GMT                   GMT         43   
           51.493847 -0.163025      GMT                   GMT         23   
2024-11-05 48.822495  2

In [40]:
(
    pd.DataFrame(json_output)
        .explode('daily')
        .reset_index(names=['variable'])
)

,variable,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,daily_units,daily
0,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-26
1,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-27
2,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-28
3,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-29
4,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-30
5,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-10-31
6,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-11-01
7,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-11-02
8,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-11-03
9,time,51.493847,-0.163025,0.154972,0,GMT,GMT,23,iso8601,2024-11-04
